<a href="https://colab.research.google.com/github/Celest14l/StudyMail-Analyser/blob/main/StudyFlow_Mail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
!pip install click
!pip install joblib
!pip install regex
!pip install tqdm
!pip install nltk==3.8.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.9 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.9.1
    Uninstalling nltk-3.9.1:
      Successfully uninstalled nltk-3.9.1


In [ ]:
import imaplib
import email
from email.header import decode_header
import nltk
from nltk.corpus import stopwords
import re


# Connecting to email account

In [ ]:
# Gmail IMAP server details
imap_host = "imap.gmail.com"
email_user = "Your Email"  # Replace with your email
email_pass = "Your App Password"  # Use your Gmail app password, not the account password, go to google setting search app password then create a app password

# Connect to the server
mail = imaplib.IMAP4_SSL(imap_host)

# Log in to the email account
mail.login(email_user, email_pass)


('OK', [b'spiderman78778@gmail.com authenticated (Success)'])

# Fetching emails from the inbox

In [ ]:
# Select the mailbox you want to access
mail.select("inbox")

# Search for all emails in the inbox
status, messages = mail.search(None, "ALL")

# Convert the messages to a list of email IDs
email_ids = messages[0].split()


Extract and analyze email content

In [ ]:
# Function to clean and preprocess email content
def clean_email_content(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)  # Remove excess spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Fetch and display the emails
emails = []
for email_id in email_ids[:10]:  # Just process the first 10 emails for now
    status, msg_data = mail.fetch(email_id, "(RFC822)")
    for response_part in msg_data:
        if isinstance(response_part, tuple):
            msg = email.message_from_bytes(response_part[1])
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                subject = subject.decode(encoding if encoding else "utf-8")

            from_ = msg.get("From")
            if msg.is_multipart():
                for part in msg.walk():
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    if "attachment" not in content_disposition:
                       body = ""  # Initialize body to an empty string

            if msg.is_multipart():
                for part in msg.walk():
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        if "attachment" not in content_disposition:
                            body = part.get_payload(decode=True).decode()
                            break
                    except:
                        pass
            else:
                try:
                    body = msg.get_payload(decode=True).decode()
                except:
                    pass

            # Clean the email content
            body = clean_email_content(body)

            emails.append({"subject": subject, "from": from_, "body": body})

# Show the first 2 emails
for email in emails[:2]:
    print(f"Subject: {email['subject']}")
    print(f"From: {email['from']}")
    print(f"Body: {email['body'][:100]}...\n")


Subject: Welcome to Adobe
From: account-noreply@adobe.com
Body:  adobe  dear peter parker you recently created an adobeid your adobeid is spiderman78778gmailcom the...

Subject: Create phenomenal photo looks on the go with presets
From: "Adobe Creative Cloud" <mail@email.adobe.com>
Body:  view web version httpttrgemailadobecomridt2699766972a6294e72a629e6p140je6b8tg2btuj2f2oyxolwcg4u88ma...



Categorize the emails based on content

In [ ]:
# Categorization function
def categorize_email(body):
    categories = {
        "Assignments": ["assignment", "homework", "due", "deadline"],
        "Exams": ["exam", "test", "quiz", "midterm"],
        "Meetings": ["meeting", "schedule", "discussion", "appointment"],
        "Personal": ["friend", "party", "chat", "family"]
    }

    for category, keywords in categories.items():
        for keyword in keywords:
            if keyword in body:
                return category
    return "Other"

# Categorize the emails
for email in emails:
    category = categorize_email(email["body"])
    email["category"] = category

# Show categorized emails
for email in emails[:2]:
    print(f"Subject: {email['subject']}")
    print(f"From: {email['from']}")
    print(f"Category: {email['category']}")
    print(f"Body: {email['body'][:100]}...\n")


Subject: Welcome to Adobe
From: account-noreply@adobe.com
Category: Other
Body:  adobe  dear peter parker you recently created an adobeid your adobeid is spiderman78778gmailcom the...

Subject: Create phenomenal photo looks on the go with presets
From: "Adobe Creative Cloud" <mail@email.adobe.com>
Category: Other
Body:  view web version httpttrgemailadobecomridt2699766972a6294e72a629e6p140je6b8tg2btuj2f2oyxolwcg4u88ma...



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

# Sample labeled data (replace with actual data)
emails_text = ["Due date for assignment", "Meeting at 10 AM", "Reminder for the exam tomorrow", "Catch up on the weekend"]
categories = ["Assignments", "Meetings", "Exams", "Personal"]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(emails_text, categories, test_size=0.2, random_state=42)

# Create a model pipeline (vectorizer + Naive Bayes classifier)
model = make_pipeline(CountVectorizer(), MultinomialNB())

# Train the model
model.fit(X_train, y_train)

# Predict category for a new email
email_to_predict = ["The assignment deadline is next week"]
predicted_category = model.predict(email_to_predict)
print(f"Predicted category: {predicted_category[0]}")


Predicted category: Assignments


# Interface and hugging face


In [ ]:
pip install gradio

In [ ]:
# Gmail IMAP server details
imap_host = "imap.gmail.com"
email_user = "your email"  # Replace with your email
email_pass = "your app pass"  # Use your Gmail app password, not the account password

# Connect to the server
mail = imaplib.IMAP4_SSL(imap_host)

# Log in to the email account
mail.login(email_user, email_pass)


# code with search functionality, save to file, and email slot tracking:

In [ ]:
import gradio as gr
from datetime import datetime, timedelta
import imaplib
import email as email_lib
from email.header import decode_header
import re
from transformers import pipeline
import csv

# Hugging Face text classification pipeline
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

# Global state
email_limit = 100
daily_emails = []
reset_time = datetime.now().replace(hour=23, minute=30, second=0)

# Reset email limit at 11:30 PM
def reset_email_limit():
    global email_limit, daily_emails
    if datetime.now() >= reset_time:
        email_limit = 100
        daily_emails = []

# Categorize email using Hugging Face
def categorize_email_hf(subject, body):
    content = f"Subject: {subject}\nBody: {body[:500]}"
    result = classifier(content)[0]
    if result['label'] == 'POSITIVE':
        return "Important"
    elif result['label'] == 'NEGATIVE':
        return "Spam"
    else:
        return "Other"

# Fetch and categorize emails
def fetch_emails(email_user, email_pass):
    global email_limit, daily_emails

    # Check daily limit
    reset_email_limit()
    if email_limit <= 0:
        return "Daily email limit reached. Try again tomorrow."

    try:
        # Connect to Gmail
        mail = imaplib.IMAP4_SSL("imap.gmail.com")
        mail.login(email_user, email_pass)

        # Select inbox
        mail.select("inbox")

        # Filter today's emails
        today = datetime.now().strftime("%d-%b-%Y")
        status, messages = mail.search(None, f'SINCE {today}')
        email_ids = messages[0].split()

        # Limit to available slots
        limit = min(len(email_ids), email_limit)
        emails = []
        for email_id in email_ids[:limit]:
            status, msg_data = mail.fetch(email_id, "(RFC822)")
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email_lib.message_from_bytes(response_part[1])

                    # Decode subject
                    subject, encoding = decode_header(msg.get("Subject"))[0]
                    if isinstance(subject, bytes):
                        subject = subject.decode(encoding if encoding else "utf-8")
                    else:
                        subject = subject or "No Subject"

                    # Get sender
                    from_ = msg.get("From") or "Unknown Sender"

                    # Extract body
                    body = None
                    if msg.is_multipart():
                        for part in msg.walk():
                            content_type = part.get_content_type()
                            if content_type == "text/plain":
                                body = part.get_payload(decode=True).decode(errors="ignore")
                                break
                            elif content_type == "text/html" and not body:
                                body = re.sub(r'<.*?>', '', part.get_payload(decode=True).decode(errors="ignore"))
                    else:
                        body = msg.get_payload(decode=True).decode(errors="ignore")

                    body = body or "No content available"

                    # Categorize with Hugging Face
                    category = categorize_email_hf(subject, body)

                    email_data = {
                        "subject": subject,
                        "from": from_,
                        "category": category,
                        "body": body[:200]  # Limit to first 200 characters
                    }
                    emails.append(email_data)
                    daily_emails.append(email_data)

        email_limit -= len(emails)
        return emails if emails else "No emails found for today."

    except Exception as e:
        return f"Error: {str(e)}"

# Save emails to file
def save_to_file():
    try:
        with open("categorized_emails.csv", "w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=["subject", "from", "category", "body"])
            writer.writeheader()
            writer.writerows(daily_emails)
        return "Emails saved to 'categorized_emails.csv'."
    except Exception as e:
        return f"Error: {str(e)}"

# Gradio UI
def email_categorizer(email, password):
    emails = fetch_emails(email, password)
    if isinstance(emails, str):
        return emails  # Return error or limit message
    return "\n\n".join(
        [f"Subject: {email['subject']}\nFrom: {email['from']}\nCategory: {email['category']}\nBody: {email['body']}" for email in emails]
    )

# Search function
def search_emails(keyword):
    results = [email for email in daily_emails if keyword.lower() in email["subject"].lower()]
    if not results:
        return "No matching emails found."
    return "\n\n".join(
        [f"Subject: {email['subject']}\nFrom: {email['from']}\nCategory: {email['category']}\nBody: {email['body']}" for email in results]
    )

# Gradio Blocks UI
with gr.Blocks() as demo:
    gr.Markdown("# Enhanced Email Categorizer with Hugging Face")
    gr.Markdown("Enter your email credentials to categorize today's emails.")

    with gr.Row():
        email_input = gr.Textbox(label="Email Address", placeholder="example@gmail.com")
        password_input = gr.Textbox(label="App Password", type="password", placeholder="Your app password")

    with gr.Row():
        categorize_button = gr.Button("Fetch and Categorize Emails")
        save_button = gr.Button("Save Emails to File")

    output_box = gr.Textbox(label="Categorized Emails", lines=20)

    with gr.Row():
        search_input = gr.Textbox(label="Search Emails by Keyword")
        search_button = gr.Button("Search")
        search_output = gr.Textbox(label="Search Results", lines=10)

    categorize_button.click(email_categorizer, inputs=[email_input, password_input], outputs=output_box)
    save_button.click(save_to_file, inputs=[], outputs=output_box)
    search_button.click(search_emails, inputs=[search_input], outputs=search_output)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2bbabb927a4663d53b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Fine Tuning

In [ ]:
!pip install transformers datasets gradio torch


In [ ]:
import pandas as pd

# Sample dataset (replace with your data)
data = {
    'body': [
        "Your subscription will expire tomorrow. Renew to continue enjoying premium features.",
        "We are excited to inform you about our upcoming event this weekend. Join us for exciting talks, workshops, and networking opportunities."
    ],
    'summary': [
        "Subscription expires tomorrow. Renew now.",
        "Upcoming event this weekend: talks, workshops, networking."
    ]
}

df = pd.DataFrame(data)
df.to_csv("train.csv", index=False)


In [ ]:
import pandas as pd

# Load the train and test CSVs
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Check the columns of both datasets
print("Train Columns:", train_df.columns)
print("Test Columns:", test_df.columns)

# Drop the extra columns from the test dataset
test_df = test_df[['body', 'summary']]  # Keep only 'body' and 'summary'

# Save the corrected files
test_df.to_csv('test_corrected.csv', index=False)


Train Columns: Index(['body', 'summary'], dtype='object')
Test Columns: Index(['subject', 'body', 'category', 'summary'], dtype='object')


In [ ]:
from datasets import load_dataset

# Load the corrected dataset
dataset = load_dataset("csv", data_files={"train": "train.csv", "test": "test_corrected.csv"})


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir="./fine_tuned_t5",  # Where the model will be saved
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    save_total_limit=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to="none",  # Disable wandb logging
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=None,
)

# Start training
trainer.train()

# Save the model and tokenizer
model.save_pretrained("/content/fine_tuned_t5")
tokenizer.save_pretrained("/content/fine_tuned_t5")


Epoch,Training Loss,Validation Loss
1,No log,9.426620
2,No log,9.056620
3,No log,8.865244


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


('/content/fine_tuned_t5/tokenizer_config.json',
 '/content/fine_tuned_t5/special_tokens_map.json',
 '/content/fine_tuned_t5/spiece.model',
 '/content/fine_tuned_t5/added_tokens.json',
 '/content/fine_tuned_t5/tokenizer.json')

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the fine-tuned model
model_path = "./fine_tuned_t5"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Summarization function
def summarize_email(email_body):
    input_text = f"summarize: {email_body}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=15, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

# Example email body
email_body = "Your subscription will expire tomorrow. Renew your subscription now to continue enjoying premium features."
summary = summarize_email(email_body)
print("Summary:", summary)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Summary: renew your subscription now to continue enjoying premium features.


In [ ]:
import gradio as gr
from datetime import datetime, timedelta
import imaplib
import email as email_lib
from email.header import decode_header
import re
from transformers import pipeline
import csv

# Hugging Face text classification pipeline for categorization
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

# Hugging Face summarization pipeline
summarizer = pipeline("summarization", model="t5-small")

# Global state
email_limit = 100
daily_emails = []
reset_time = datetime.now().replace(hour=23, minute=30, second=0)

# Reset email limit at 11:30 PM
def reset_email_limit():
    global email_limit, daily_emails
    if datetime.now() >= reset_time:
        email_limit = 100
        daily_emails = []

# Categorize email using Hugging Face
def categorize_email_hf(subject, body):
    content = f"Subject: {subject}\nBody: {body[:500]}"
    result = classifier(content)[0]
    if result['label'] == 'POSITIVE':
        return "Important"
    elif result['label'] == 'NEGATIVE':
        return "Spam"
    else:
        return "Other"

# Summarize body content if it's more than 20 words
# Max length for the model input (tokens)
MAX_LENGTH = 512

# Summarize the body content if it's more than 20 words or too long
def summarize_body(body):
    words = body.split()

    # If the body has more than 20 words, summarize
    if len(words) > 20:
        # Truncate or split if body is too long
        if len(body.split()) > 1000:  # If the body has more than 1000 words, split it
            chunks = [body[i:i+MAX_LENGTH] for i in range(0, len(body), MAX_LENGTH)]
            summary = ''
            for chunk in chunks:
                summary += summarizer(chunk, max_length=50, min_length=25, do_sample=False)[0]['summary_text']
            return summary
        else:
            # Summarize using Hugging Face summarizer
            summary = summarizer(body, max_length=50, min_length=25, do_sample=False)[0]['summary_text']
            return summary
    else:
        # If the body is short, return as is
        return body


# Fetch and categorize emails
def fetch_emails(email_user, email_pass):
    global email_limit, daily_emails

    # Check daily limit
    reset_email_limit()
    if email_limit <= 0:
        return "Daily email limit reached. Try again tomorrow."

    try:
        # Connect to Gmail
        mail = imaplib.IMAP4_SSL("imap.gmail.com")
        mail.login(email_user, email_pass)

        # Select inbox
        mail.select("inbox")

        # Filter today's emails
        today = datetime.now().strftime("%d-%b-%Y")
        status, messages = mail.search(None, f'SINCE {today}')
        email_ids = messages[0].split()

        # Limit to available slots
        limit = min(len(email_ids), email_limit)
        emails = []
        for email_id in email_ids[:limit]:
            status, msg_data = mail.fetch(email_id, "(RFC822)")
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email_lib.message_from_bytes(response_part[1])

                    # Decode subject
                    subject, encoding = decode_header(msg.get("Subject"))[0]
                    if isinstance(subject, bytes):
                        subject = subject.decode(encoding if encoding else "utf-8")
                    else:
                        subject = subject or "No Subject"

                    # Get sender
                    from_ = msg.get("From") or "Unknown Sender"

                    # Extract body
                    body = None
                    if msg.is_multipart():
                        for part in msg.walk():
                            content_type = part.get_content_type()
                            if content_type == "text/plain":
                                body = part.get_payload(decode=True).decode(errors="ignore")
                                break
                            elif content_type == "text/html" and not body:
                                body = re.sub(r'<.*?>', '', part.get_payload(decode=True).decode(errors="ignore"))
                    else:
                        body = msg.get_payload(decode=True).decode(errors="ignore")

                    body = body or "No content available"

                    # Summarize body if more than 20 words
                    body = summarize_body(body)

                    # Categorize with Hugging Face
                    category = categorize_email_hf(subject, body)

                    email_data = {
                        "subject": subject,
                        "from": from_,
                        "category": category,
                        "body": body[:200]  # Limit to first 200 characters for display
                    }
                    emails.append(email_data)
                    daily_emails.append(email_data)

        email_limit -= len(emails)
        return emails if emails else "No emails found for today."

    except Exception as e:
        return f"Error: {str(e)}"

# Save emails to file
def save_to_file():
    try:
        with open("categorized_emails.csv", "w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=["subject", "from", "category", "body"])
            writer.writeheader()
            writer.writerows(daily_emails)
        return "Emails saved to 'categorized_emails.csv'."
    except Exception as e:
        return f"Error: {str(e)}"

# Gradio UI
def email_categorizer(email, password):
    emails = fetch_emails(email, password)
    if isinstance(emails, str):
        return emails  # Return error or limit message
    return "\n\n".join(
        [f"Subject: {email['subject']}\nFrom: {email['from']}\nCategory: {email['category']}\nBody: {email['body']}" for email in emails]
    )

# Search function
def search_emails(keyword):
    results = [email for email in daily_emails if keyword.lower() in email["subject"].lower()]
    if not results:
        return "No matching emails found."
    return "\n\n".join(
        [f"Subject: {email['subject']}\nFrom: {email['from']}\nCategory: {email['category']}\nBody: {email['body']}" for email in results]
    )

# Gradio Blocks UI
with gr.Blocks() as demo:
    gr.Markdown("# Enhanced Email Categorizer with Hugging Face")
    gr.Markdown("Enter your email credentials to categorize today's emails.")

    with gr.Row():
        email_input = gr.Textbox(label="Email Address", placeholder="example@gmail.com")
        password_input = gr.Textbox(label="App Password", type="password", placeholder="Your app password")

    with gr.Row():
        categorize_button = gr.Button("Fetch and Categorize Emails")
        save_button = gr.Button("Save Emails to File")

    output_box = gr.Textbox(label="Categorized Emails", lines=20)

    with gr.Row():
        search_input = gr.Textbox(label="Search Emails by Keyword")
        search_button = gr.Button("Search")
        search_output = gr.Textbox(label="Search Results", lines=10)

    categorize_button.click(email_categorizer, inputs=[email_input, password_input], outputs=output_box)
    save_button.click(save_to_file, inputs=[], outputs=output_box)
    search_button.click(search_emails, inputs=[search_input], outputs=search_output)

demo.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b0ebf1255fa75a9ec2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Token indices sequence length is longer than the specified maximum sequence length for this model (4563 > 512). Running this sequence through the model will result in indexing errors


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://b0ebf1255fa75a9ec2.gradio.live
